In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn import decomposition
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import time

import umap

In [0]:
pip install git+https://github.com/samueljackson92/coranking.git

  Cloning https://github.com/samueljackson92/coranking.git to /tmp/pip-req-build-tw5b5_af
  Running command git clone -q https://github.com/samueljackson92/coranking.git /tmp/pip-req-build-tw5b5_af
  Created wheel for coranking: filename=coranking-0.1.1-cp36-cp36m-linux_x86_64.whl size=87031 sha256=581f26009280dec818d3caab214972470da9b7a9941c6afb95e34242b19f0427
  Stored in directory: /tmp/pip-ephem-wheel-cache-zrbcj40i/wheels/8e/e0/11/4ae911b6fc472b8bf6087e29d861d2b798c119603adf4b93fa
Successfully built coranking


In [0]:
import coranking
from coranking.metrics import trustworthiness, continuity, LCMC


###1. Loading and Processing MNIST

In [0]:
link1='https://drive.google.com/open?id=1LAXOvFLyMmK8Pkdw6efWhGHtIQpkGnYd'

fluff, id = link1.split('=')
print (id) # Verify that you have everything after '='

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train.csv') 
training_data  = pd.read_csv('train.csv')

1LAXOvFLyMmK8Pkdw6efWhGHtIQpkGnYd


In [0]:
labels  = training_data['label']
training_data = training_data.drop("label",axis=1)

In [0]:
print(training_data.shape)

(42000, 784)


In [0]:
from sklearn.preprocessing import StandardScaler
standardized_data = StandardScaler().fit_transform(training_data)
print(standardized_data.shape)

(42000, 784)


In [0]:
covar_matrix = np.matmul(standardized_data.T , standardized_data)

print ( "The shape of variance matrix = ", covar_matrix.shape)

The shape of variance matrix =  (784, 784)


In [0]:
training_data

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


###Decision Tree on Original Data

In [0]:
from sklearn import tree

X_train, X_test, y_train, y_test = train_test_split(training_data[0:9999], labels[0:9999], test_size=0.25, random_state=49)

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.792

###UMAP

In [0]:

reducer = umap.UMAP(random_state=42,n_components=16)
embedding = reducer.fit_transform(training_data[:100])

In [0]:
embedding.shape

(100, 16)

###Decision T on UMAP


In [0]:
from prettytable import PrettyTable
t_Umap_10k = PrettyTable(['DT_score_UmapData', 'Dimensions', 'Umap_time','DecisionTree_time' ,'total_time', 'trust_Umap','cont_Umap' ,'lcmc_Umap'])
t_Umap_392dims = PrettyTable(['DT_score_origData','DT_time_origData','DT_score_UmapData', 'Dimensions', 'Umap_time','DecisionTree_time' ,'total_time', 'trust_Umap','cont_Umap' ,'lcmc_Umap'])

def MDS_Decision_Tree (training_data, components,size):

  #Dt on training{:size}
  X_train, X_test, y_train, y_test = train_test_split(training_data[:size], labels[:size], test_size=0.25, random_state=49)
  clf_start_time = time.time()
  clf = tree.DecisionTreeClassifier()
  clf = clf.fit(X_train, y_train)
  clf_time0 = time.time() - clf_start_time
  clf_time0 = str(round(clf_time0, 4))
  score0=clf.score(X_test,y_test)
  score0 = str(round(score0, 4))



  #Reducing to mentioned components
  mds_start_time = time.time()
  reducer = umap.UMAP(random_state=42,n_components=components)
  training_data_mds = reducer.fit_transform(training_data[:size])  
  mds_time = time.time() - mds_start_time
  
  
  #Applying Decision Tree
  X_train, X_test, y_train, y_test = train_test_split(training_data_mds, labels[:size], test_size=0.25, random_state=49)
  clf_start_time = time.time()
  clf = tree.DecisionTreeClassifier()
  clf = clf.fit(X_train, y_train)
  clf_time = time.time() - clf_start_time
  score=clf.score(X_test,y_test)
  score = str(round(score, 4))

  total_time = mds_time+clf_time
  mds_time = str(round(mds_time, 4))
  clf_time = str(round(clf_time, 4))
  total_time = str(round(total_time, 4))

  Q = coranking.coranking_matrix(training_data[:size], training_data_mds)
  trust_mds = trustworthiness(Q, min_k=1, max_k=50).sum()/50
  cont_mds = continuity(Q, min_k=1, max_k=50).sum()/50
  lcmc_mds = LCMC(Q, min_k=1, max_k=50).sum()/50

  trust_mds = str(round(trust_mds, 4))
  cont_mds = str(round(cont_mds, 4))
  lcmc_mds = str(round(lcmc_mds, 4))

  print(size)
  t_Umap_392dims.add_row([score0,clf_time0,score, components, mds_time, clf_time ,total_time, trust_mds,cont_mds ,lcmc_mds])
  return

In [0]:
dimensions=784
size=9999
while dimensions >= 2:
  MDS_Decision_Tree (training_data, dimensions,size)
  dimensions=int(dimensions/2)

print(t_Umap_10k)

+-------------------+------------+-----------+-------------------+------------+------------+-----------+-----------+
| DT_score_UmapData | Dimensions | Umap_time | DecisionTree_time | total_time | trust_Umap | cont_Umap | lcmc_Umap |
+-------------------+------------+-----------+-------------------+------------+------------+-----------+-----------+
|       0.9208      |    784     | 2835.8126 |      13.5803      |  2849.393  |   0.9623   |   0.951   |   0.3862  |
|       0.9232      |    392     | 1194.6021 |       6.5282      | 1201.1303  |   0.9623   |   0.9511  |   0.3864  |
|       0.9208      |    196     |  563.2642 |       3.0488      |  566.313   |   0.9624   |   0.9512  |   0.3857  |
|       0.9224      |     98     |  234.5255 |       1.4656      |  235.9911  |   0.9623   |   0.951   |   0.386   |
|       0.9208      |     49     |  152.6641 |       0.7715      |  153.4356  |   0.9625   |   0.951   |   0.3859  |
|       0.9148      |     24     |  116.8647 |       0.3176     

In [0]:
dimensions=392
size=1000
while size <= 10000:
  MDS_Decision_Tree (training_data, dimensions,size)
  size=size+1000

print(t_Umap_392dims)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
+-------------------+------------------+-------------------+------------+-----------+-------------------+------------+------------+-----------+-----------+
| DT_score_origData | DT_time_origData | DT_score_UmapData | Dimensions | Umap_time | DecisionTree_time | total_time | trust_Umap | cont_Umap | lcmc_Umap |
+-------------------+------------------+-------------------+------------+-----------+-------------------+------------+------------+-----------+-----------+
|       0.604       |      0.0933      |       0.812       |    392     |  54.7035  |       0.2614      |   54.965   |   0.9278   |   0.913   |   0.484   |
|       0.676       |      0.2112      |       0.868       |    392     |  119.9298 |       0.702       |  120.6318  |   0.9427   |   0.9304  |   0.4655  |
|       0.7027      |      0.3521      |       0.8693      |    392     |  194.9941 |       1.0742      |  196.0683  |   0.9499   |   0.9365  |   0.4523  |
|       0.694